# An Introduction to Association Rules in Python

##### Association rules is a rule-based learning method used to draw frequent patterns and correlations from datasets such as transactional and relational data.

##### In essence it computes the co-occurence statistics between items, in the form of an implication expression (X → Y).

##### For instance, in customer basket analysis, {diaper} → {beer} means if diaper is bought, then beer is put into basket.

#### 4 fundamental concepts in association rules:

* *(Not a Rule)* Support: number of times X occurs over all instances. 

* Support(X→Y) is the probability of co-occurence of both items within all data.

* Confidence(X→Y) is the probability of Y occurs given that X is present.

* Lift(X→Y) is the probability of Y being bought given that X is present, taking into account the popularity of Y as well.

* Conviction(X→Y) is the measure of implication. A value > 1 indicates that Y is highly depending on X.

So basically it is probability/statistics. A simple but useful decision making tool for a wide range of usages such as market basket analysis, customer relationship management, recommender system, marketing activities, network traffic analysis, intrusion detection (fraud & malware detection) and bioinformatics.


# Example 1

### Before getting into the formnulas and terminology, let's begin by a simple example.

Mlxtend is a rich and useful library for machine learning. It provides methods in association rules with a major algorithm *apriori*.

You can install mlxtend via pip or conda.

In [ ]:
# !pip install mlxtend

In [1]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

To use association rules, first we neeed some data in one-hot encoded format.

Imagine in a grocery database, there are order id with some products...

In [2]:
data = {'ID':[1,2,3,4,5,6],
       'Onion':[1,0,0,1,1,1],
       'Potato':[1,1,0,1,1,1],
       'Burger':[1,1,0,0,1,1],
       'Milk':[0,1,1,1,0,1],
       'Beer':[0,0,1,0,1,0]}

In [3]:
df = pd.DataFrame(data)

In [4]:
df = df[['ID', 'Onion', 'Potato', 'Burger', 'Milk', 'Beer' ]]

In [5]:
df

,ID,Onion,Potato,Burger,Milk,Beer
0,1,1,1,1,0,0
1,2,0,1,1,1,0
2,3,0,0,0,1,1
3,4,1,1,0,1,0
4,5,1,1,1,0,1
5,6,1,1,1,1,0


### Then, we can generate frequent itemsets based on *support*.

Here we need to set the minimum support value between [0,1]. Using min_supp = 50% means we only want itemsets that co-occur more than half of the time.

`apriori(df, min_support=0.5, use_colnames=False, max_len=None)`

In [10]:
frequent_itemsets = apriori(df[['Onion', 'Potato', 'Burger', 'Milk', 'Beer' ]], min_support=0.5, use_colnames=True)

In [11]:
frequent_itemsets

,support,itemsets
0,0.666667,(Onion)
1,0.833333,(Potato)
2,0.666667,(Burger)
3,0.666667,(Milk)
4,0.666667,"(Onion, Potato)"
5,0.500000,"(Onion, Burger)"
6,0.666667,"(Potato, Burger)"
7,0.500000,"(Potato, Milk)"
8,0.500000,"(Onion, Burger, Potato)"


Itemsets with 1, 2 or 3 items are returned, with support > 0.5

The only itemset with 3 products is [Onion, Potato, Burger].

### Final Step: generate the rules with their corresponding support, confidence and lift, (and leverage & conviction):

```association_rules(df, metric='confidence', min_threshold=0.8)```

* Here, df means the frequent_itemsets dataframe; 

* metrics is the parameters to consider if there is association. You can set it to one of the five metrics.

* min_threshold is the mininum value for the specified metrics.

In [23]:
rules = association_rules(frequent_itemsets, metric='lift', min_threshold=1)

In [24]:
df

,ID,Onion,Potato,Burger,Milk,Beer
0,1,1,1,1,0,0
1,2,0,1,1,1,0
2,3,0,0,0,1,1
3,4,1,1,0,1,0
4,5,1,1,1,0,1
5,6,1,1,1,1,0


In [25]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Onion),(Potato),0.666667,0.833333,0.666667,1.00,1.200,0.111111,inf
1,(Potato),(Onion),0.833333,0.666667,0.666667,0.80,1.200,0.111111,1.666667
2,(Onion),(Burger),0.666667,0.666667,0.500000,0.75,1.125,0.055556,1.333333
3,(Burger),(Onion),0.666667,0.666667,0.500000,0.75,1.125,0.055556,1.333333
4,(Potato),(Burger),0.833333,0.666667,0.666667,0.80,1.200,0.111111,1.666667
5,(Burger),(Potato),0.666667,0.833333,0.666667,1.00,1.200,0.111111,inf
6,"(Onion, Burger)",(Potato),0.500000,0.833333,0.500000,1.00,1.200,0.083333,inf
7,"(Onion, Potato)",(Burger),0.666667,0.666667,0.500000,0.75,1.125,0.055556,1.333333
8,"(Potato, Burger)",(Onion),0.666667,0.666667,0.500000,0.75,1.125,0.055556,1.333333
9,(Onion),"(Potato, Burger)",0.666667,0.666667,0.500000,0.75,1.125,0.055556,1.333333


### Intrepreting the result:

We can see that there are quite a few rules with a high lift value which means that it occurs more frequently than would be expected given the number of transaction and product combinations.

Several are high in confidence as well. But domain knowledge will be useful in explaining the phenomenon.

In [16]:
rules [ (rules['lift'] >1.125)  & (rules['confidence']> 0.8)  ]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Onion),(Potato),0.666667,0.833333,0.666667,1.0,1.2,0.111111,inf
5,(Burger),(Potato),0.666667,0.833333,0.666667,1.0,1.2,0.111111,inf
6,"(Onion, Burger)",(Potato),0.500000,0.833333,0.500000,1.0,1.2,0.083333,inf


Subsetting the lift and confidence values return you with the itemsets that are relatively highly correlated in this data.

We can see that:

* **If Onion or Burger is in a users' basket, it is highly likely that the user will buy Potato as well.**
* **If Burger and Onion is in a users' basket, it is highly likely that the user will also buy Potato.**

### Some notes on Lift, Conviction & Leverage:


1.  Lift(X→Y) : the likelihood of Y being bought when X is present, taking into account the popularity of Y as well.
    > When Lift=1,  X makes no impact on Y  
    > When Lift>1, there is a relationship between X & Y
2.  Conviction(X→Y): Conviction is a measure of the implication and has value 1 if items are unrelated.
    > A high conviction value means that the consequent is highly depending on the antecedent. For instance, in the case of a perfect confidence score, the denominator becomes 0 (due to 1 - 1) for which the conviction score is defined as 'inf'. Similar to lift, if items are independent, the conviction is 1.
3.  Leverage(X→Y): the difference between the observed frequency of X and Y appearing together and the frequency that would be expected if X and Y were independent. An leverage value of 0 indicates independence.

# Example 2

In [26]:
retail_shopping_basket = {'ID':[1,2,3,4,5,6],
                         'Basket':[['Beer', 'Diaper', 'Pretzels', 'Chips', 'Aspirin'],
                                   ['Diaper', 'Beer', 'Chips', 'Lotion', 'Juice', 'BabyFood', 'Milk'],
                                   ['Soda', 'Chips', 'Milk'],
                                   ['Soup', 'Beer', 'Diaper', 'Milk', 'IceCream'],
                                   ['Soda', 'Coffee', 'Milk', 'Bread'],
                                   ['Beer', 'Chips']
                                  ]
                         }

In [27]:
retail = pd.DataFrame(retail_shopping_basket)
retail

,ID,Basket
0,1,"[Beer, Diaper, Pretzels, Chips, Aspirin]"
1,2,"[Diaper, Beer, Chips, Lotion, Juice, BabyFood,..."
2,3,"[Soda, Chips, Milk]"
3,4,"[Soup, Beer, Diaper, Milk, IceCream]"
4,5,"[Soda, Coffee, Milk, Bread]"
5,6,"[Beer, Chips]"


In [28]:
retail = retail[['ID', 'Basket']]

In [29]:
pd.options.display.max_colwidth=100

Suppose we have a list of customer ids to a list of basket items:

In [30]:
retail

,ID,Basket
0,1,"[Beer, Diaper, Pretzels, Chips, Aspirin]"
1,2,"[Diaper, Beer, Chips, Lotion, Juice, BabyFood, Milk]"
2,3,"[Soda, Chips, Milk]"
3,4,"[Soup, Beer, Diaper, Milk, IceCream]"
4,5,"[Soda, Coffee, Milk, Bread]"
5,6,"[Beer, Chips]"


First one-hot encode the basket, but how?

In [31]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
retail = pd.DataFrame(mlb.fit_transform(retail.Basket), columns=mlb.classes_)

In [32]:
retail

,Aspirin,BabyFood,Beer,Bread,Chips,Coffee,Diaper,IceCream,Juice,Lotion,Milk,Pretzels,Soda,Soup
0,1,0,1,0,1,0,1,0,0,0,0,1,0,0
1,0,1,1,0,1,0,1,0,1,1,1,0,0,0
2,0,0,0,0,1,0,0,0,0,0,1,0,1,0
3,0,0,1,0,0,0,1,1,0,0,1,0,0,1
4,0,0,0,1,0,1,0,0,0,0,1,0,1,0
5,0,0,1,0,1,0,0,0,0,0,0,0,0,0


Making use of `Series.str.get_dummies`, we can easily encode lists of items in a dataframe's column!

In [33]:
frequent_itemsets_2 = apriori(retail, min_support=0.2,use_colnames=True)

In [34]:
frequent_itemsets_2

,support,itemsets
0,0.666667,(Beer)
1,0.666667,(Chips)
2,0.500000,(Diaper)
3,0.666667,(Milk)
4,0.333333,(Soda)
5,0.500000,"(Beer, Chips)"
6,0.500000,"(Diaper, Beer)"
7,0.333333,"(Beer, Milk)"
8,0.333333,"(Diaper, Chips)"
9,0.333333,"(Milk, Chips)"


Just by calculating the support(X>Y), [Beer, Chips] & [Beer, Diaper] are the two frequent basket of intereseted.

But which one is more correlated than the other?

In [36]:
association_rules(frequent_itemsets_2, metric='lift',min_threshold=1.5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Diaper),(Beer),0.500000,0.666667,0.500000,1.000000,1.5,0.166667,inf
1,(Beer),(Diaper),0.666667,0.500000,0.500000,0.750000,1.5,0.166667,2.000000
2,(Milk),(Soda),0.666667,0.333333,0.333333,0.500000,1.5,0.111111,1.333333
3,(Soda),(Milk),0.333333,0.666667,0.333333,1.000000,1.5,0.111111,inf
4,"(Diaper, Chips)",(Beer),0.333333,0.666667,0.333333,1.000000,1.5,0.111111,inf
5,(Beer),"(Diaper, Chips)",0.666667,0.333333,0.333333,0.500000,1.5,0.111111,1.333333
6,"(Diaper, Milk)",(Beer),0.333333,0.666667,0.333333,1.000000,1.5,0.111111,inf
7,"(Beer, Milk)",(Diaper),0.333333,0.500000,0.333333,1.000000,2.0,0.166667,inf
8,(Diaper),"(Beer, Milk)",0.500000,0.333333,0.333333,0.666667,2.0,0.166667,2.000000
9,(Beer),"(Diaper, Milk)",0.666667,0.333333,0.333333,0.500000,1.5,0.111111,1.333333



Clearly, {Diaper, Beer} is the most associated itemset in this data!

# Example 3 - Movie Genre Associations

It seems a bit boring playing only with basket analysis and imaginary datasets.

In this example, let's play with an open dataset [MovieLens (small)](https://grouplens.org/datasets/movielens/).

This dataset (ml-latest-small) describes 5-star rating and free-text tagging activity from MovieLens, a movie recommendation service. It contains 100004 ratings and 1296 tag applications across 9125 movies. These data were created by 671 users between January 09, 1995 and October 16, 2016.

Users were selected at random for inclusion. All selected users had rated at least 20 movies. No demographic information is included. Each user is represented by an id, and no other information is provided.

We might want to take a look at the data and look at the stat first:

In [37]:
from google.colab import drive
drive.mount('/content/drive')

movies = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Intellipaat/Weekend_ML_Batch/Association_Rule/movies.csv")

# movies = pd.read_csv('ml-latest-small/movies.csv')

Mounted at /content/drive


In [38]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [40]:
movies_ohe = movies.drop('genres',1).join(movies.genres.str.get_dummies())

<ipython-input-40-f56a92c69fe3>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  movies_ohe = movies.drop('genres',1).join(movies.genres.str.get_dummies())


In [41]:
movies_ohe.head()

,movieId,title,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),0,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale (1995),0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
4,5,Father of the Bride Part II (1995),0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
movies_ohe.shape

(9125, 22)

### Let's get back to analysing the genre associations:

In [43]:
movies_ohe.set_index(['movieId','title'],inplace=True)
movies_ohe

,,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movieId,title,,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,Jumanji (1995),0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,Grumpier Old Men (1995),0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,Waiting to Exhale (1995),0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0
5,Father of the Bride Part II (1995),0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162672,Mohenjo Daro (2016),0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
163056,Shin Godzilla (2016),0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0
163949,The Beatles: Eight Days a Week - The Touring Years (2016),0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [54]:
movies_ohe.Children.value_counts(normalize=True)

0    0.93611
1    0.06389
Name: Children, dtype: float64

In [44]:
movies_ohe.shape

(9125, 20)

In [55]:
frequent_itemsets_movies = apriori(movies_ohe,use_colnames=True,min_support=0.01)

In [56]:
frequent_itemsets_movies

,support,itemsets
0,0.169315,(Action)
1,0.122411,(Adventure)
2,0.048986,(Animation)
3,0.063890,(Children)
4,0.363288,(Comedy)
...,...,...
87,0.010301,"(Mystery, Crime, Drama)"
88,0.032000,"(Crime, Drama, Thriller)"
89,0.012493,"(Mystery, Crime, Thriller)"
90,0.010411,"(Horror, Drama, Thriller)"


In [59]:
rules_movies =  association_rules(frequent_itemsets_movies, metric='lift', min_threshold=3)

In [63]:
  rules_movies.sort_values('lift',ascending=False).head(20)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
28,"(Children, Adventure)",(Animation),0.029260,0.048986,0.014685,0.501873,10.245163,0.013252,1.909178
33,(Animation),"(Children, Adventure)",0.048986,0.029260,0.014685,0.299776,10.245163,0.013252,1.386328
30,"(Adventure, Animation)",(Children),0.023342,0.063890,0.014685,0.629108,9.846673,0.013194,2.523941
31,(Children),"(Adventure, Animation)",0.063890,0.023342,0.014685,0.229846,9.846673,0.013194,1.268132
49,"(Animation, Comedy)",(Children),0.021370,0.063890,0.013370,0.625641,9.792409,0.012005,2.500567
50,(Children),"(Animation, Comedy)",0.063890,0.021370,0.013370,0.209262,9.792409,0.012005,1.237617
8,(Children),(Animation),0.063890,0.048986,0.027068,0.423671,8.648758,0.023939,1.650122
9,(Animation),(Children),0.048986,0.063890,0.027068,0.552573,8.648758,0.023939,2.092205
51,(Animation),"(Children, Comedy)",0.048986,0.032877,0.013370,0.272931,8.301641,0.011759,1.330166
48,"(Children, Comedy)",(Animation),0.032877,0.048986,0.013370,0.406667,8.301641,0.011759,1.602832


***As we can see in this dataset, the support and hence confidence values are fairly small. This makes it difficult interpreting the result based on these two values. Whereas, the lift and conviction remains to very intuitive and representative. That is why we should understand the meaning of all of the 5 metrics to accurately interpret the result!***

In [64]:
rules_movies[(rules_movies.lift>10)]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
28,"(Children, Adventure)",(Animation),0.029260,0.048986,0.014685,0.501873,10.245163,0.013252,1.909178
33,(Animation),"(Children, Adventure)",0.048986,0.029260,0.014685,0.299776,10.245163,0.013252,1.386328


* As we are expecting the {Romance, Drama} pair, it is not as correlated as other groups such as {Animation, Childres} which has a much higher lift & conviction levels.

In [66]:
rules_movies[(rules_movies.lift>3)].sort_values(by=['lift','confidence'], ascending=[False,False])

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
28,"(Children, Adventure)",(Animation),0.029260,0.048986,0.014685,0.501873,10.245163,0.013252,1.909178
33,(Animation),"(Children, Adventure)",0.048986,0.029260,0.014685,0.299776,10.245163,0.013252,1.386328
30,"(Adventure, Animation)",(Children),0.023342,0.063890,0.014685,0.629108,9.846673,0.013194,2.523941
31,(Children),"(Adventure, Animation)",0.063890,0.023342,0.014685,0.229846,9.846673,0.013194,1.268132
49,"(Animation, Comedy)",(Children),0.021370,0.063890,0.013370,0.625641,9.792409,0.012005,2.500567
...,...,...,...,...,...,...,...,...,...
58,"(Horror, Drama)",(Thriller),0.017644,0.189479,0.010411,0.590062,3.114122,0.007068,1.977179
60,"(Drama, Mystery)",(Thriller),0.031671,0.189479,0.018301,0.577855,3.049696,0.012300,1.920004
63,(Thriller),"(Drama, Mystery)",0.189479,0.031671,0.018301,0.096588,3.049696,0.012300,1.071857
52,"(Drama, Thriller)",(Crime),0.087123,0.120548,0.032000,0.367296,3.046884,0.021497,1.389989


By making a subset with ordering with lift & conviction:

* The highest correlation: {Animation, Childres} correlates in both directions! Recall those Pixar & Disney films that we love watching
* {Children, Adventure} ...
* {Fantasy, Adventure} ... How to interpret these two pairs?

The best way is to go back to your movies table and check it out!

In [ ]:
pd.options.display.max_rows=50

So we want Adventure & Children but NOT Animation...

In [67]:
movies[ (movies.genres.str.contains('Children')) & (~movies.genres.str.contains('Animation'))]

,movieId,title,genres
1,2,Jumanji (1995),Adventure|Children|Fantasy
7,8,Tom and Huck (1995),Adventure|Children
26,27,Now and Then (1995),Children|Drama
32,34,Babe (1995),Children|Drama
36,38,It Takes Two (1995),Children|Comedy
...,...,...,...
8918,135268,Zenon: Z3 (2004),Adventure|Children|Comedy
8960,139620,Everything's Gonna Be Great (1998),Adventure|Children|Comedy|Drama
8967,140152,Dreamcatcher (2015),Children|Crime|Documentary
8981,140747,16 Wishes (2010),Children|Drama|Fantasy


So, well, what are these movies? I rarely know any of them... (proves again the notion that domain knowledge is of utmost importance in data science!)

Viola, I know ***Tomorrowland (2015)***! We all know this movie, so we sort of understand why {Children, Adventure} is an associated pair. Given that this is not an animation, but its interesting and fantasy storyline in discovering the secrets of a mystic place kind of succeeded in targeting little boys and girls.

There are more to discover. Try finding an interesting pair on your own!

# Summary

To recap, a straightforward 4-steps approach to association rule:

1. One-hot encone the basket in dataframe.
2. Generate frequent itemsets using `apriori`.
3. Generate rule with `association_rules`.
4. Interpret & evalute the result with metrics.

### References:
1. [Introduction to Market Basket Analysis in Python](http://pbpython.com/market-basket-analysis.html)
2. [Movie genre associations](https://mathematicaforprediction.wordpress.com/2013/10/06/movie-genre-associations/)
3. [Mining Association Rules](https://paginas.fe.up.pt/~ec/files_0506/slides/04_AssociationRules.pdf)
4. [Association Rules Generation from Frequent Itemsets](https://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/)
5. F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets: History and Context. ACM Transactions on Interactive Intelligent Systems (TiiS) 5, 4, Article 19 (December 2015), 19 pages. DOI=http://dx.doi.org/10.1145/2827872